In [9]:
from typing import Any
from langchain import LLMChain

import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate

In [10]:
user_mindmap_prompt = """
    In a markdown file (MD) plan out a mind map on the topic {topic}. Follow the format in this example. Write it in a code snippet I can copy from directly. Provide only code, no description. Include the exact format I used below.
Example markdown format:

# Central topic

## Main topic 1

### Subtopic 1

– Subtopic 1

– Subtopic 2

– Subtopic 3

### Subtopic 2

– Subtopic 1

– Subtopic 2

– Subtopic 3

## Main topic 2

### Subtopic 1

– Subtopic 1

– Subtopic 2

– Subtopic 3"""

user_translate_prompt = """
Übersetze den folgenden Text in {language}. Beinhalte die Markdown Formatierung bei.

Text: 
{brainstorm}"""

In [11]:
def getBrainstormPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["topic"], template=user_mindmap_prompt)

def getTranslationPrompt() -> PromptTemplate:
    return PromptTemplate(input_variables=["language", "brainstorm"], template=user_translate_prompt)


In [21]:
import os
os.environ['http_proxy'] = "http://internet-proxy-client.muenchen.de:80" 
os.environ['https_proxy'] = "http://internet-proxy-client.muenchen.de:80" 

In [22]:
openai.api_type = "azure"
openai.api_base = f"https://cog-tb726faeoukc2.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = "9324d0a0d0bd4d72b0e34596a9dc4a9c"

In [18]:
verbose = True
llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    temperature= 0.9,
    max_tokens=4096,
    n=1,
    deployment_name= "chat",
    openai_api_key=openai.api_key,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    openai_api_type=openai.api_type
)
brainstormChain = LLMChain(llm=llm, prompt=getBrainstormPrompt(), output_key="brainstorm", verbose=verbose)
translationChain = LLMChain(llm=llm, prompt=getTranslationPrompt(), output_key="translation", verbose=verbose)
overall_chain = SequentialChain(
    chains=[brainstormChain, translationChain], 
    input_variables=["language", "topic"],
    output_variables=["brainstorm","translation"],
    verbose=verbose)

In [19]:
topic = "Dinge, die man auf dem Oktoberfest machen kann"
language="englisch"

In [23]:
result = await overall_chain.acall({"topic": topic, "language": language})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    In a markdown file (MD) plan out a mind map on the topic Dinge, die man auf dem Oktoberfest machen kann. Follow the format in this example. Write it in a code snippet I can copy from directly. Provide only code, no description. Include the exact format I used below.
Example markdown format:

# Central topic

## Main topic 1

### Subtopic 1

– Subtopic 1

– Subtopic 2

– Subtopic 3

### Subtopic 2

– Subtopic 1

– Subtopic 2

– Subtopic 3

## Main topic 2

### Subtopic 1

– Subtopic 1

– Subtopic 2

– Subtopic 3

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Übersetze den folgenden Text in englisch. Beinhalte die Markdown Formatierung bei.

Text: 
# Dinge, die man auf dem Oktoberfest machen kann

## Essen und Trinken

### Essen

- Hendl
- Brezen
- Schweinshaxe
- Radi
- Obatzda
- Leberkäse
- Steckerlfisch

### Trinken

- Bier
- Radler
- Spezi
- Wein


In [52]:
result

{'topic': 'Dinge, die man auf dem Oktoberfest machen kann',
 'language': 'englisch',
 'brainstorm': '# Dinge, die man auf dem Oktoberfest machen kann\n\n## Traditionelle Aktivitäten\n\n### Auf der Wiesn spazieren\n\n- Über das Gelände schlendern\n- Die vielen Zelte besuchen\n- Die bunten Lichter genießen\n\n### Bier trinken\n\n- Eine Maß Bier trinken\n- Das Bier schunkelnd genießen\n- Mit Freunden anstoßen\n\n### Bayern erleben\n\n- Tracht tragen\n- Bayerns Kultur erleben\n- In einem der vielen Festzüge mitlaufen\n\n## Essensmöglichkeiten\n\n### Leckereien probieren\n\n- Hendl essen\n- Brezn naschen\n- Bayerischen Käse genießen\n\n### Süße Verführung\n\n- Kaiserschmarrn essen\n- Apfelstrudel probieren\n- Schokofrüchte genießen\n\n### Fischspezialitäten\n\n- Steckerlfisch essen\n- Gebratene Garnelen probieren\n- Sushi genießen\n\n## Interessante Veranstaltungen\n\n### Fahrgeschäfte\n\n- Karussells fahren\n- Achterbahn fahren\n- Über den Dächern Münchens chillen\n\n### Musikalisches Prog

In [45]:
result["translation"]

'# Things to do at Oktoberfest\n\n## Food and Drink\n\n### Food\n\n- Roast chicken\n- Pretzel\n- Pork knuckle\n- Meatloaf\n- Cheese spread\n\n### Drinks\n\n- Beer\n- Wheat beer\n- Shandy\n- Apple juice with sparkling water\n- Non-alcoholic beer\n\n## Rides\n\n### Classics\n\n- Ferris wheel\n- Ghost train\n- Bumper cars\n\n### Thrill rides\n\n- Looping roller coaster\n- Free fall tower\n- Breakdance ride\n\n## Music and Entertainment\n\n### Music\n\n- Brass bands\n- DJs\n- Live bands\n\n### Games and Prizes\n\n- Tin can toss\n- Shooting gallery\n- Duck fishing\n\n## Tradition and Culture\n\n### Tracht (traditional clothing)\n\n- Lederhosen (leather shorts)\n- Dirndl (traditional dress)\n\n### Traditional events\n\n- Costume parade\n- Oktoberfest run\n- Tapping of the first beer barrel'